<table align="left">
  <tr>
    <td><img src="Museum.png" alt="ice cream truck in front of Louvre" width="120"/></td>
    <td align="left"><h1>Lesson 5: Building a Deep Research-like agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Set up and test search tools

In [1]:
from tavily import TavilyClient
from smolagents import tool, Tool
import os
from dotenv import load_dotenv

load_dotenv(override=True) # load variables from local .env file

@tool
def web_search(query: str) -> str:
    """Searches the web for your query.

    Args:
        query: Your query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query)
    return str(response["results"])


class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, 40000)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

print(web_search("What is the melting temperature of vanilla ice cream in °C?"))

[{'url': 'https://en.wikipedia.org/wiki/Ice_cream', 'title': 'Ice cream - Wikipedia', 'content': 'The result is a smooth, semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). ... Vanilla ice cream served with whipped cream, chocolate', 'score': 0.7992077, 'raw_content': None}, {'url': 'https://www.icecreamscience.com/blog/vanilla-bean-ice-cream-recipe', 'title': 'Vanilla Bean Ice Cream - Recipe', 'content': 'Once the temperature reaches 75°C (167°F), turn the heat down to low, move your pan about 1/4 of the way off the heat, and continue heating', 'score': 0.78552294, 'raw_content': None}, {'url': 'http://scienceline.ucsb.edu/getkey.php?key=6870', 'title': 'UCSB Science Line', 'content': 'When this happens, some of the methyl cellulose molecules stick to each other instead, and this makes the mixture take on a gel-like consistency (the process is called **"thermogelation,"** meaning that temperature controls the process of forming a gel.)  Now, to answer your q

## Set up an agent

In [2]:
from PIL import Image

from smolagents import CodeAgent, OpenAIServerModel

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

code_model = "gpt-4.1-mini"

model = OpenAIServerModel(
    #"gpt-4o",
    code_model,  #updated model
    max_completion_tokens=8096,
)

<div style="font-size: 20px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> In the notebook, we reduced the museum count from 10 to 3 to reduce runtime and cost.  
  This will reduce the number of ice cream trucks you will need to buy to start your business!
</div>


In [3]:
#changed to 2024 to avoid decaying internet content
request_museums = """
Could you give me a sorted list of the top 3 museums in the world in 2024,
along with their visitor count (in millions) that year, and the approximate daily temperature
in July at their location ?"""

In [4]:
agent = CodeAgent(
    model=model,
    tools=[web_search, VisitWebpageTool()],
    max_steps=10
)
agent.logger.console.width=66
result = agent.run(request_museums)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_museums_2024 = web_search(query="Top 3 most visited         
  museums in the world 2024 with visitor counts")                 
  print(top_museums_2024)                                         
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'url': 
'https://www.wanderlustmagazine.com/news/most-visited-museums-2024
/', 'title': 'These were the most-visited museums around the world
in 2024', 'content': "In second place, with 6.8 million visitors 
was the Vatican Museums, followed closely by London's British 
Museum with 6.4 million guests. The", 'score': 0.9693272, 
'raw_content': None}, {'url': 
'https://www.theartnewspaper.com/2025/04/01/the-worlds-most-visite
d-museums-2024-', 'title': "Exclusive | The world's most-visited 
museums 2024", 'content': 'Meanwhile, the Natural History Museum, 
not included in our survey, received 6.3 million and had a record 
number of visitors in its 2023-24 financial year. In Lisbon, the 
MAC/CCB Museum of Contemporary Art had its first full year, 
welcoming 589,000 people; however, this was a drop from the 
million visitors its predecessor at the site, the Museu Coleção 
Berardo, achieved in 2019. Of course, the very existence of the 
National Museum of African American History and Culture could be 
argued as political—and, with its consistent 1.6 million visitors 
both last year and in 2023, it is the fourth most popular art 
museum in the country, and the second most visited in the nation’s
capital. In third place, after the National Gallery of Art, is the
Museum of Modern Art (MoMA) with 2.7 million visitors (down 6% on 
2023).', 'score': 0.9557879, 'raw_content': None}, {'url': 
'https://tourstravelfinder.com/most-visited-museums-in-the-world/?
srsltid=AfmBOoqMuWhJL8v_6lCGITcp3Y1y4g0R15EIfkx9dLpnZGvqr3r3-zK4',
 'title': 'Most Visited Museums in the World: Criteria, Trends & 
Insights in 2024', 'content': 'Annually the Louvre in Paris, 
France, attracts more than 9.6 million visitors, which makes it 
one of the most visited museums in the world.', 'score': 
0.9392538, 'raw_content': None}, {'url': 
'https://www.brandvm.com/post/most-visited-museums-world', 
'title': 'Top 10 Most Visited Museums in the World - Brand Vision 
Marketing', 'content': 'Millions of travelers flock to the **most 
visited museums in the world** each year, eager to experience art,
history, and culture up close. From the world’s **most visited 
museum** – the legendary Louvre in Paris – to must-see 
institutions in New York and Beijing, each destination offers a 
unique journey for visitors. London’s **Natural History Museum** 
is the world’s most visited science/natural history museum, 
attracting roughly **6.3 million visitors in 2024**. As a major 
cultural attraction in one of China’s former capitals, the Nanjing
Museum offers travelers deep insight into Chinese civilization and
consistently ranks among **the most visited museums in the world**
for its rich and diverse collections.', 'score': 0.92359936, 
'raw_content': None}, {'url': 
'https://www.travelandtourworld.com/news/article/top-10-most-visit
ed-museums-around-the-world-reveal-surging-global-tourism-record-b
reaking-attendance-and-the-growing-challenges-of-overcrowding/', 
'title': 'Top 10 Most Visited Museums Around the World Reveal 
Surging ...', 'content': 'With a collection spanning ancient 
civilisations to modern art, The Met welcomed nearly 6 million 
visitors in 2024. Its expansive Fifth Avenue', 'score': 0.8364614,
'raw_content': None}]

Out: None

[Step 1: Duration 4.01 seconds| Input tokens: 2,097 | Output 
tokens: 155]

...

[Step 2: Duration 7.59 seconds| Input tokens: 5,267 | Output 
tokens: 348]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_3_museums = [                                               
      {"name": "Louvre", "location": "Paris, France",             
  "visitors_millions": 9.6, "july_temp_c": 25.5},                 
      {"name": "Vatican Museums", "location": "Vatican City",     
  "visitors_millions": 6.8, "july_temp_c": 25},                   
      {"name": "British Museum", "location": "London, UK",        
  "visitors_millions": 6.4, "july_temp_c": 22},                   
  ]                                                               
                                                                  
  # Sorted already in descending order by visitors, so no need    
  to reorder                                                      
  print(top_3_museums)                                            
  final_answer(top_3_museums)                                     
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'name': 'Louvre', 'location': 'Paris, France', 
'visitors_millions': 9.6, 'july_temp_c': 25.5}, {'name': 'Vatican 
Museums', 'location': 'Vatican City', 'visitors_millions': 6.8, 
'july_temp_c': 25}, {'name': 'British Museum', 'location': 
'London, UK', 'visitors_millions': 6.4, 'july_temp_c': 22}]

Out - Final answer: [{'name': 'Louvre', 'location': 'Paris, 
France', 'visitors_millions': 9.6, 'july_temp_c': 25.5}, {'name': 
'Vatican Museums', 'location': 'Vatican City', 
'visitors_millions': 6.8, 'july_temp_c': 25}, {'name': 'British 
Museum', 'location': 'London, UK', 'visitors_millions': 6.4, 
'july_temp_c': 22}]

[Step 3: Duration 4.12 seconds| Input tokens: 11,947 | Output 
tokens: 625]

In [5]:
#modified code to handle case where agent doesn't return a python list.
import pandas as pd

try:
    display(pd.DataFrame(result))
except Exception as e:
    print("Could not display as DataFrame:", e)
    print(result)

,name,location,visitors_millions,july_temp_c
0,Louvre,"Paris, France",9.6,25.5
1,Vatican Museums,Vatican City,6.8,25.0
2,British Museum,"London, UK",6.4,22.0


## Create a multi-agent researcher
### Start with web search agent

In [6]:
web_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,  # updated model
        max_completion_tokens=8096,
    ),
    tools=[web_search, VisitWebpageTool()],
    max_steps=10,
    name="web_agent",
    description="Runs web searches for you."
)
web_agent.logger.console.width=66

### Setup manager agent

In [7]:
from smolagents import Tool
from typing import Any
from smolagents.utils import make_image_url, encode_image_base64

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel(
        "gpt-4o",
    )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
        "Also, any run that invents numbers should fail."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,
        max_tokens=8096,
    ),
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",  #added
        "plotly.express.colors",  #added
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)
manager_agent.logger.console.width=66

In [8]:
manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: ['geopandas', 'plotly', 
│   'plotly.express', 'plotly.express.colors', 'shapely', 'json', 
│   'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description          ┃ Arguments            ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final     │ answer (`any`): The  │
│   │              │ answer to the given  │ final answer to the  │
│   │              │ problem.             │ problem              │
│   └──────────────┴──────────────────────┴──────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description      ┃ Arguments       ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Searches the web │ query           │
            │               │ for your query.  │ (`string`):     │
            │               │                  │ Your query      │
            │ visit_webpage │ Visits a webpage │ url (`string`): │
            │               │ at the given url │ The url of the  │
            │               │ and reads its    │ webpage to      │
            │               │ content as a     │ visit.          │
            │               │ markdown string. │                 │
            │               │ Use this to      │                 │
            │               │ browse webpages. │                 │
            │ final_answer  │ Provides a final │ answer (`any`): │
            │               │ answer to the    │ The final       │
            │               │ given problem.   │ answer to the   │
            │               │                  │ problem         │
            └───────────────┴──────────────────┴─────────────────┘

In [9]:
# added this code, remove plot file if exits.
import os
os.path.exists("saved_map.png") and os.remove("saved_map.png")

False

In [ ]:
manager_agent.run(f"""
{request_museums}

Then make me a spatial map of the world using px.scatter_map, with the biggest museums 
represented as scatter points of size depending on visitor count and color depending 
on the average temperature in July.
Save the map to saved_map.png, then return it!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",
     color_continuous_scale=px.colors.sequential.Magma_r, size_max=15, zoom=1)
fig.show()
final_answer(fig)

Do not invent any numbers! You must only use numbers sourced from the internet.
""")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
│ Then make me a spatial map of the world using px.scatter_map,  │
│ with the biggest museums                                       │
│ represented as scatter points of size depending on visitor     │
│ count and color depending                                      │
│ on the average temperature in July.                            │
│ Save the map to saved_map.png, then return it!                 │
│                                                                │
│ Here's an example of how to plot and return a map:             │
│ import plotly.express as px                                    │
│ df = px.data.carshare()                                        │
│ fig = px.scatter_map(df, lat="centroid_lat",                   │
│ lon="centroid_lon", text="name", color="peak_hour",            │
│      color_continuous_scale=px.colors.sequential.Magma_r,      │
│ size_max=15, zoom=1)                                           │
│ fig.show()                                                     │
│ final_answer(fig)                                              │
│                                                                │
│ Do not invent any numbers! You must only use numbers sourced   │
│ from the internet.                                             │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

────────────────────────── Initial plan ──────────────────────────
Here are the facts I know and the plan of action that I will 
follow to solve the task:
```
---
## Facts survey
### 1.1. Facts given in the task
- The task is to produce a sorted list of the top 3 museums in the
world in 2024, ranked by visitor count.
- For each museum, we need the visitor count (in millions) for 
2024.
- For each museum, we need the approximate daily temperature in 
July at the museum's location.
- We need to create a spatial map using Plotly Express's 
px.scatter_map, where:
  - Scatter points represent the biggest museums.
  - Point size depends on visitor count.
  - Point color depends on the average temperature in July.
- The final map should be saved as `saved_map.png` and returned.
- An example code snippet is provided for a map using 
px.scatter_map.

### 1.2. Facts to look up
- The current top 3 museums worldwide ranked by visitor count 
specifically for the year 2024, including their visitor numbers in
millions.
  *Source:* Recent reputable museum attendance reports for 2024, 
official museum websites, cultural/statistics organizations, or 
verified summary articles online.
- Locations (latitude and longitude) of these top 3 museums.
  *Source:* Official museum websites or reliable geographical 
databases.
- Approximate average daily temperature in July at each museum's 
location.
  *Source:* Climate data websites such as World Weather Online, 
NOAA, Weather.com, or historical climate data services that 
provide average temperatures by city or region.

### 1.3. Facts to derive
- Sorting museums by visitor count to determine the top 3.
- Converting visitor counts to a consistent unit (millions) if 
necessary.
- Extracting or calculating latitudes and longitudes for the 
museums.
- Mapping the average July temperature to appropriate color values
for plotting.
- Determining scatter point size scale based on visitor counts.
- Creating a DataFrame integrating museum names, visitor counts, 
lat/lon, and July temperature.
- Generating and saving the plotly scatter_map image as 
`saved_map.png`.

## Plan
1. Conduct a web search to obtain the list of top museums in the 
world by visitor count for 2024, including visitor numbers.
2. Retrieve the geographical locations (latitude and longitude) of
these museums.
3. Collect approximate average July temperature data for each 
museum location.
4. Verify and format all collected data consistently.
5. Sort the museums by visitor count and select the top 3.
6. Prepare a data structure (e.g., pandas DataFrame) containing: 
museum name, visitor count (millions), latitude, longitude, and 
July temperature.
7. Use plotly.express px.scatter_map to create the spatial map 
with:
   - Point size scaled by visitor count,
   - Point color scaled by July temperature,
   - Proper annotations (museum names).
8. Save the map figure to `saved_map.png`.
9. Return the saved image using the `final_answer` tool.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I need to find the current top museums in the world for  
2024 ranked by visitor numbers. I will do a web search to retrieve
the visitor counts and locations for these museums.               
                                                                  
Code:                                                             
```py                                                             
task = "Find the current top museums in the world for 2024 ranked 
by annual visitor numbers, including their visitor counts in      
millions, and their city or address."                             
result = web_agent(task=task)                                     
print(result)                                                     
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  task = "Find the current top museums in the world for 2024      
  ranked by annual visitor numbers, including their visitor       
  counts in millions, and their city or address."                 
  result = web_agent(task=task)                                   
  print(result)                                                   
 ────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  # Extracting key museums and their visitor numbers and cities   
  from the gathered text for interim summary                      
  top_museums_2024 = [                                            
      {"name": "Musée du Louvre", "visitors_millions": 8.7,       
  "city": "Paris, France"},                                       
      {"name": "Vatican Museum", "visitors_millions": 6.8,        
  "city": "Vatican City"},                                        
      {"name": "British Museum", "visitors_millions": 6.5,        
  "city": "London, UK"},                                          
      {"name": "Natural History Museum", "visitors_millions":     
  6.3, "city": "London, UK"},                                     
      {"name": "Tate Modern", "visitors_millions": 4.6, "city":   
  "London, UK"},                                                  
      {"name": "Shanghai Museum East", "visitors_millions": 4.3,  
  "city": "Shanghai, China"},                                     
      {"name": "Art Gallery of New South Wales",                  
  "visitors_millions": 2.4, "city": "Sydney, Australia"},         
      {"name": "Louvre Abu Dhabi", "visitors_millions": 1.4,      
  "city": "Abu Dhabi, UAE"},                                      
      {"name": "M+", "visitors_millions": 2.6, "city": "Hong      
  Kong"},                                                         
      {"name": "Hong Kong Palace Museum", "visitors_millions":    
  None, "city": "Hong Kong"},                                     
  ]                                                               
                                                                  
  print(top_museums_2024)                                         
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'name': 'Musée du Louvre', 'visitors_millions': 8.7, 'city': 
'Paris, France'}, {'name': 'Vatican Museum', 'visitors_millions': 
6.8, 'city': 'Vatican City'}, {'name': 'British Museum', 
'visitors_millions': 6.5, 'city': 'London, UK'}, {'name': 'Natural
History Museum', 'visitors_millions': 6.3, 'city': 'London, UK'}, 
{'name': 'Tate Modern', 'visitors_millions': 4.6, 'city': 'London,
UK'}, {'name': 'Shanghai Museum East', 'visitors_millions': 4.3, 
'city': 'Shanghai, China'}, {'name': 'Art Gallery of New South 
Wales', 'visitors_millions': 2.4, 'city': 'Sydney, Australia'}, 
{'name': 'Louvre Abu Dhabi', 'visitors_millions': 1.4, 'city': 
'Abu Dhabi, UAE'}, {'name': 'M+', 'visitors_millions': 2.6, 
'city': 'Hong Kong'}, {'name': 'Hong Kong Palace Museum', 
'visitors_millions': None, 'city': 'Hong Kong'}]

Out: None

[Step 8: Duration 6.79 seconds| Input tokens: 42,935 | Output 
tokens: 1,538]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

> Note, the map will look a bit different as we are only searching for 3 locations.

In [ ]:
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)
fig.show()